In [2]:
import numpy as np
from pathlib import Path
import shutil
from natsort import natsorted
import cv2
import imutils

from helper import COCOToYOLOBox, YOLOToCOCOBox, img2label_paths, getLabelPaths
import xmltodict
from analyser import process_misdetections, video_to_images, copyFiles

%reload_ext autoreload
%autoreload 2

# Group into Folders

In [ ]:
root_dir = Path("datasets/raw/misdetections/misdetection_output_210324/")

for folder in root_dir.glob("*"):
    if folder.is_dir():
        process_misdetections(folder)

# Gather Videos

In [ ]:
# root_dir = Path("datasets/raw/pooltests/210324")
# vid_ext = "*.mp4"

root_dir = Path("validation/pooltests/210324")
vid_ext = "*.avi"

vid_paths = list(root_dir.rglob(vid_ext))
vid_dir = root_dir / "videos"
vid_dir.mkdir(exist_ok=True)
for vid_path in vid_paths:
    shutil.copy2(
        vid_path,
        vid_dir
        / f"{vid_path.parent.parent.stem}_{vid_path.parent.stem}_{vid_path.name}",
    )

# Split Video into N Parts

In [ ]:
dir_name = "pooltests/120324/21.bag"
vid_name = Path("bottom.rect.image.mp4")
dir_path = Path(f"datasets/raw/{dir_name}/")
vid_path = dir_path / vid_name

vid_cap = cv2.VideoCapture(str(vid_path))

N = 3
DISPLAY_FLAG = True
frame_count = int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
bounds = [[i * frame_count // N, (i + 1) * frame_count // N] for i in range(N)]

for i in range(N):
    out_vid_path = dir_path / f"{vid_name.stem}_split_{i}.mp4"

    out_vid = cv2.VideoWriter(
        str(out_vid_path),
        cv2.VideoWriter_fourcc(*"mp4v"),
        vid_cap.get(cv2.CAP_PROP_FPS),
        (
            int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        ),
    )

    vid_cap.set(cv2.CAP_PROP_POS_FRAMES, bounds[i][0])

    while vid_cap.isOpened():
        ret, frame = vid_cap.read()
        if not ret:
            break

        if bounds[i][0] <= vid_cap.get(cv2.CAP_PROP_POS_FRAMES) < bounds[i][1]:
            out_vid.write(frame)

            if DISPLAY_FLAG:
                frame = imutils.resize(frame, height=500)
                cv2.imshow("Ground Truth", frame)
                key = cv2.waitKey(5)
                if key == ord("q") or key == ord("Q"):
                    break

                if key == ord("p") or key == ord("P"):
                    while True:
                        key = cv2.waitKey(0)
                        if key == ord("p") or key == ord("P"):
                            break


cv2.destroyAllWindows()
vid_cap.release()
out_vid.release()

# Video to Images

In [ ]:
# dirName = "input"
# videoName = "input.mp4"
# SUBSAMPLE = 1

# dirName = "pooltests/271223/main_gate_detection"
# videoName = "left.image_raw.mp4"
# SUBSAMPLE = 5

# dirName = "pooltests/040124/qual_2.bag"
# videoName = "left.raw.mp4"
# SUBSAMPLE = 5
# frameBounds = [[890, 3180]]

# dirName = "pooltests/040124/qual_2.bag/left.raw"
# videoName = "left.raw.mp4"
# SUBSAMPLE = 5
# frameBounds = [[0, 10000]]

# dirName = "pooltests/040124/qual_3.bag/right.raw"
# videoName = "right.raw.mp4"
# SUBSAMPLE = 5
# frameBounds = [[300, 2200]]

# dirName = "pooltests/110124/qual_gate_1.bag"
# videoName = "left.raw.mp4"
# SUBSAMPLE = 5
# frameBounds = [[435, 3300]]

# dirName = "pooltests/110124/mixed_2.bag"
# videoName = "right.raw.mp4"
# SUBSAMPLE = 1
# frameBounds = [[435, 3300]]

# dirName = "pooltests/180124/2.bag"
# videoName = "left.compressed.mp4"
# SUBSAMPLE = 10
# frameBounds = [[0, 6900]]

# dirName = "pooltests/180124/3.bag"
# videoName = "left.compressed.mp4"
# SUBSAMPLE = 10
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/180124/4.bag"
# videoName = "left.compressed.mp4"
# SUBSAMPLE = 5
# frameBounds = [[0, 3215]]

# dirName = "pooltests/180124/5.bag"
# videoName = "left.compressed.mp4"
# SUBSAMPLE = 5
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/180124/9.bag"
# videoName = "left.compressed.mp4"
# SUBSAMPLE = 5
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/180124/10.bag"
# videoName = "left.compressed.mp4"
# SUBSAMPLE = 2
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/260124/3.bag"
# videoName = "left.compressed.mp4"
# SUBSAMPLE = 4
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/260124/5.bag"
# videoName = "left.compressed.mp4"
# SUBSAMPLE = 4
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/300124/6.bag"
# videoName = "bag6.mp4"
# SUBSAMPLE = 4
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/300124/10.bag"
# videoName = "bag10.mp4"
# SUBSAMPLE = 4
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/300124/14.bag"
# videoName = "bag14.mp4"
# SUBSAMPLE = 4
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/270224/1.bag"
# videoName = "output.mp4"
# SUBSAMPLE = 4
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/270224/2.bag"
# videoName = "output.mp4"
# SUBSAMPLE = 2
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/270224/4.bag"
# videoName = "output.mp4"
# SUBSAMPLE = 2
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/120324/21.bag/bottom.rect.image_split_0"
# videoName = "bottom.rect.image_split_0.mp4"
# SUBSAMPLE = 4
# frameBounds = [[0, 1e6]]

# dirName = "pooltests/120324/21.bag/bottom.rect.image_split_1"
# videoName = "bottom.rect.image_split_1.mp4"
# SUBSAMPLE = 4
# frameBounds = [[0, 1e6]]


dirName = "pooltests/120324/21.bag/bottom.rect.image_split_2"
videoName = "bottom.rect.image_split_2.mp4"
SUBSAMPLE = 4
frameBounds = [[0, 1e6]]

dirPath = Path(f"datasets/raw/{dirName}/")
video_to_images(dirPath, videoName, subsample=SUBSAMPLE, frame_bounds=frameBounds)

In [ ]:
root_dir = Path("datasets/raw/misdetections/misdetection_output_210324/")

subsample = 2
frame_bounds = [[0, 1e6]]  # Take all frames

for bag_dir in natsorted(root_dir.glob("*")):
    for dir_path in natsorted(bag_dir.glob("*")):
        print(dir_path)
        video_to_images(
            dirPath=dir_path,
            videoName=dir_path.stem + "_raw.mp4",
            subsample=subsample,
            frame_bounds=frame_bounds,
            display_interval=20,
        )

# Convert

## old-bags

In [ ]:
saveDir = Path("datasets/processed/old-bags/")
datasetDir = Path("datasets/raw/old-bags")
imagePaths = list(datasetDir.glob("*.[jp][pn]g"))

imgRootDir = saveDir / "images" / "root"
labelRootDir = saveDir / "labels" / "root"
imgRootDir.mkdir(parents=True, exist_ok=True)
labelRootDir.mkdir(parents=True, exist_ok=True)

for imagePath in natsorted(imagePaths):
    shutil.copy2(imagePath, imgRootDir / imagePath.name)

    oldLabelPath = (datasetDir / imagePath.stem).with_suffix(".xml")
    annotation = xmltodict.parse(open(oldLabelPath).read())
    detections = [list(map(int, annotation["annotation"]["object"]["bndbox"].values()))]
    detections = [[d[0], d[1], d[2] - d[0], d[3] - d[1]] for d in detections]

    for d in detections:
        img = cv2.imread(str(imagePath))
        x, y, w, h = d

        cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (0, 0, 255), 3)

    img = imutils.resize(img, height=500)
    cv2.imshow("Ground Truth", img)
    key = cv2.waitKey(10)
    if key == ord("q") or key == ord("Q"):
        break

    detections = map(lambda d: COCOToYOLOBox(d, img.shape[-2::-1]), detections)
    detections = [["0"] + list(map(str, d)) for d in detections]
    detections = [" ".join(d) + "\n" for d in detections]
    f = open((saveDir / "labels" / "root" / oldLabelPath.stem).with_suffix(".txt"), "w")
    f.writelines(detections)
    f.close()

cv2.destroyAllWindows()

## ICBJ-v2i-yolov8, dataset-ai-v1i, sauvc_v2_bboxes_yolo_front

Data is already in YOLOv8 format, but need to remap classes.

Remap the following:

icbj-v2i-yolov8  
['Blue_Drum', 'Main_Gate', 'Red_Flare', 'Yellow_Flare']

dataset-ai-v1i  
['bola', 'ember biru', 'ember merah', 'flare biru', 'flare kuning', 'flare merah', 'gawang', 'karpet hijau']  
['ball', 'blue bucket', 'red bucket', 'blue flare', 'yellow flare', 'red flare', 'gate',  'green carpet'] 

sauvc_v2_bboxes_yolo  
- 0: qualification_gate_side
- 1: qualification_gate
- 2: gate_left
- 3: gate_right
- 4: gate
- 5: red_bucket
- 6: blue_bucket
- 7: orange_flare
- 8: yellow_flare
- 9: red_flare
- 10: blue_flare
- 11: mat
- 12: ball
- 13: pinger

To the following:
- 0: gate
- 1: orange-flare
- 2: blue-flare
- 3: red-flare
- 4: yellow-flare
- 5: blue-drum
- 6: red-drum

In [ ]:
datasetName = "sauvc_v2_bboxes_yolo_front"
remap = {"4": "0", "5": "6", "6": "5", "7": "1", "8": "4", "9": "3", "10": "2"}

# datasetName = "icbj-v2i-yolov8"
# remap = {
#     '0': '5',
#     '1': '0',
#     '2': '3',
#     '3': '4'
# }

# datasetName = "dataset-ai-v1i"
# remap = {
#     '1': '5',
#     '2': '6',
#     '3': '2',
#     '4': '4',
#     '5': '3',
#     '6': '0'
# }

# datasetName = "icbj-v2i-yolov8-cvat"
# remap = {"1": "1", "2": "2", "3": "3", "4": "4", "5": "5", "6": "6"}

In [ ]:
saveDir = Path(f"datasets/processed/{datasetName}/")
datasetDir = Path(f"datasets/raw/{datasetName}")
imgPaths = natsorted(list(datasetDir.rglob("*.[jp][pn]g")))
labelPaths = natsorted(img2label_paths(list(map(str, imgPaths))))

# Make save directories
imgRootDir = saveDir / "images" / "root"
labelRootDir = saveDir / "labels" / "root"
imgRootDir.mkdir(parents=True, exist_ok=True)
labelRootDir.mkdir(parents=True, exist_ok=True)

for imgPath, labelPath in zip(imgPaths, labelPaths):
    # Skip if label does not exist
    if not Path(labelPath).exists():
        continue

    # Copy image only if there is a label
    shutil.copy2(imgPath, imgRootDir / imgPath.name)

    # Get old detections
    with open(labelPath, "r") as f:
        fStr = f.read()

    detections = [detection.split(" ") for detection in fStr.strip().split("\n")]
    new_detections = []

    for detection in detections:
        cls = detection[0]
        if cls not in remap:
            continue

        new_detection = detection.copy()
        new_detection[0] = remap[cls]
        new_detections.append(new_detection)

    # Convert to new detection
    new_detections = [" ".join(d) + "\n" for d in new_detections]
    f = open(
        (saveDir / "labels" / "root" / Path(labelPath).stem).as_posix() + ".txt", "w"
    )
    f.writelines(new_detections)
    f.close()

## sauvc_v2_bboxes_yolo

Extract only the images for the front camera

In [ ]:
# import os

# bad_label_paths = list(
#     map(str, Path("datasets/raw/sauvc_v2_bboxes_yolo_front/images").glob("*.jpg"))
# )

# bad_image_paths = np.char.replace(bad_label_paths, "images", "labels")
# bad_image_paths = np.char.replace(bad_image_paths, ".jpg", ".txt")

# for image_path in bad_image_paths:
#     if not os.path.exists(image_path):
#         open(str(np.char.replace(image_path, "labels/", "labels/keep/")), "w").write("")

#     else:
#         shutil.move(
#             str(image_path), str(np.char.replace(image_path, "labels/", "labels/keep/"))
#         )

Generate train file for export into CVAT

In [ ]:
# img_dir = Path("datasets/raw/sauvc_v2_bboxes_yolo_front/images")

# image_paths = list(Path("datasets/raw/sauvc_v2_bboxes_yolo_front/images").glob("*.jpg"))

# f = open(img_dir.parent / "train.txt", "w")

# for image_path in image_paths:
#     f.write(f"data/obj_train_data/{str(image_path.name)}\n")

## gate-jedy8, auv-project

Subsample roboflow datasets

Remove images with segmentation labels for `auv-project-quali-gate`

In [ ]:
bad_label_paths = list(
    map(str, Path("datasets/raw/auv-project-quali-gate/valid/bad_labels").glob("*.txt"))
)

bad_image_paths = np.char.replace(bad_label_paths, "bad_labels", "images")
bad_image_paths = np.char.replace(bad_image_paths, ".txt", ".jpg")

# DANGER
# for image_path in bad_image_paths:
#     Path(image_path).unlink()

In [ ]:
def replace_paths(paths):
    paths = np.char.replace(paths, "datasets/raw/", "datasets/processed/")
    paths = np.char.replace(paths, "/train/", "/root/")
    paths = np.char.replace(paths, "/valid/", "/root/")
    paths = np.char.replace(paths, "/test/", "/root/")
    paths = np.char.replace(paths, "/root/images/", "/images/root/")
    paths = np.char.replace(paths, "/root/labels/", "/labels/root/")
    return paths


# dir_name = "gate-jedy8"
# subsample = 4

# dir_name = "auv-project-gate-sample"
# subsample = 4

dir_name = "auv-project-quali-gate"
subsample = 4

save_dir = Path(f"datasets/processed/{dir_name}/")
save_image_dir = save_dir / "images/root"
save_label_dir = save_dir / "labels/root"
save_image_dir.mkdir(parents=True, exist_ok=True)
save_label_dir.mkdir(parents=True, exist_ok=True)

image_paths = np.array(
    list(map(str, Path(f"datasets/raw/{dir_name}/").glob("**/*.jpg")))
)

np.random.seed(314159)
image_paths = np.random.choice(image_paths, round(len(image_paths) / 4), replace=False)

label_paths = getLabelPaths(image_paths)

save_image_paths = replace_paths(image_paths)
save_label_paths = replace_paths(label_paths)

for src, dst in zip(image_paths, save_image_paths):
    shutil.copy2(src, dst)

for src, dst in zip(label_paths, save_label_paths):
    shutil.copy2(src, dst)

# Check Conversion

In [ ]:
# dirName = "icbj-v2i-yolov8-cvat"
# dirName = "old-bags"
# dirName = "dataset-ai-v1i"
# dirName = "gate-jedy8"
# dirName = "auv-project-gate-sample"
# dirName = "auv-project-quali-gate"
# dirName = "input"
dirName = "sauvc_v2_bboxes_yolo_front_cvat"
# dirName = "pooltests/040124/qual_2.bag"
# dirName = "pooltests/040124/qual_3.bag/right.raw"
# dirName = "pooltests/110124/qual_gate_1.bag"

dirPath = Path(f"datasets/processed/{dirName}")
imgPaths = natsorted(dirPath.glob("images/root/*"))

assert len(imgPaths) == len(list(dirPath.glob("labels/root/*")))

for imgPath in imgPaths:
    img = cv2.imread(str(imgPath))
    labelPath = (
        str(imgPath)
        .replace("images", "labels")
        .replace(".jpg", ".txt")
        .replace(".png", ".txt")
    )

    f = open(labelPath, "r")
    boxes = f.read().strip().split("\n")

    classes = [int(box.strip().split()[0]) for box in boxes if box != ""]
    boxes = [
        [float(num) for num in box.strip().split()[1:]] for box in boxes if box != ""
    ]

    for box, cls in zip(boxes, classes):
        if box != []:
            x, y, w, h = YOLOToCOCOBox(box, img.shape[-2::-1])
            cv2.rectangle(
                img,
                (int(x), int(y)),
                (int(x + w), int(y + h)),
                colors_bgr[cls],
                1,
            )

    img = imutils.resize(img, height=500)
    cv2.imshow("Ground Truth", img)
    key = cv2.waitKey(50)

    if key == ord("p") or key == ord("P"):
        while True:
            key = cv2.waitKey(0)
            if key == ord("p") or key == ord("P"):
                break

    if key == ord("q") or key == ord("Q"):
        break

cv2.destroyAllWindows()

# Split Training, Validation, Test

In [3]:
default_proportion = {"valid": 0.2, "test": 0.0}
proportions = {
    "old-bags": {"valid": 0.2, "test": 0.1},
    "icbj-v2i-yolov8-cvat": default_proportion,
    "gate-jedy8": default_proportion,
    "auv-project-gate-sample": default_proportion,
    "auv-project-quali-gate": default_proportion,
    "input": default_proportion,
    "sauvc_v2_bboxes_yolo_front_cvat": default_proportion,
    "pooltests/271223/main_gate_detection": default_proportion,
    "pooltests/040124/qual_2.bag": default_proportion,
    "pooltests/040124/qual_3.bag/right.raw": default_proportion,
    "pooltests/110124/mixed_2.bag": default_proportion,
    "pooltests/110124/qual_gate_1.bag": default_proportion,
    "pooltests/180124/2.bag": default_proportion,
    "pooltests/180124/3.bag": default_proportion,
    "pooltests/180124/4.bag": default_proportion,
    "pooltests/180124/5.bag": default_proportion,
    "pooltests/180124/9.bag": default_proportion,
    "pooltests/180124/10.bag": default_proportion,
    "pooltests/300124/6.bag": default_proportion,
    "pooltests/300124/10.bag": default_proportion,
    "pooltests/300124/14.bag": default_proportion,
    "pooltests/270224/1.bag": default_proportion,
    "pooltests/270224/2.bag": default_proportion,
    "pooltests/270224/4.bag": default_proportion,
    "pooltests/120324/21.bag/bottom.rect.image_split_0": default_proportion,
    "pooltests/120324/21.bag/bottom.rect.image_split_1": default_proportion,
}

In [4]:
# Add misdetection dataset dirs
misdetection_dir_paths = [
    list(bag_dir.glob("*"))
    for bag_dir in Path(
        "datasets/processed/misdetections/misdetection_output_210324"
    ).glob("*")
]
misdetection_dir_paths = [path for paths in misdetection_dir_paths for path in paths]
misdetection_dir_names = natsorted(
    ["/".join(path.parts[2:]) for path in misdetection_dir_paths]
)

for dir_name in misdetection_dir_names:
    proportions[dir_name] = default_proportion

proportions

{'old-bags': {'valid': 0.2, 'test': 0.1},
 'icbj-v2i-yolov8-cvat': {'valid': 0.2, 'test': 0.0},
 'gate-jedy8': {'valid': 0.2, 'test': 0.0},
 'auv-project-gate-sample': {'valid': 0.2, 'test': 0.0},
 'auv-project-quali-gate': {'valid': 0.2, 'test': 0.0},
 'input': {'valid': 0.2, 'test': 0.0},
 'sauvc_v2_bboxes_yolo_front_cvat': {'valid': 0.2, 'test': 0.0},
 'pooltests/271223/main_gate_detection': {'valid': 0.2, 'test': 0.0},
 'pooltests/040124/qual_2.bag': {'valid': 0.2, 'test': 0.0},
 'pooltests/040124/qual_3.bag/right.raw': {'valid': 0.2, 'test': 0.0},
 'pooltests/110124/mixed_2.bag': {'valid': 0.2, 'test': 0.0},
 'pooltests/110124/qual_gate_1.bag': {'valid': 0.2, 'test': 0.0},
 'pooltests/180124/2.bag': {'valid': 0.2, 'test': 0.0},
 'pooltests/180124/3.bag': {'valid': 0.2, 'test': 0.0},
 'pooltests/180124/4.bag': {'valid': 0.2, 'test': 0.0},
 'pooltests/180124/5.bag': {'valid': 0.2, 'test': 0.0},
 'pooltests/180124/9.bag': {'valid': 0.2, 'test': 0.0},
 'pooltests/180124/10.bag': {'val

In [6]:
trainSizes, validSizes, testSizes = [], [], []
dirNames = list(proportions.keys())

for dirName in dirNames:
    validProp = proportions[dirName]["valid"]
    testProp = proportions[dirName]["test"]

    rootDir = Path(f"datasets/processed/{dirName}/images/root")

    imagePaths = np.array(natsorted(rootDir.glob("*")))

    # Reset each time so that number of processed dirs before does not
    # affect choice.
    np.random.seed(314159)
    validAndTest = np.random.choice(
        imagePaths, int(len(imagePaths) * (validProp + testProp)), replace=False
    )
    valid = np.random.choice(
        validAndTest, int(len(imagePaths) * validProp), replace=False
    )
    test = validAndTest[~np.isin(validAndTest, valid)]
    train = imagePaths[~np.isin(imagePaths, validAndTest)]

    trainLabels = getLabelPaths(train)
    validLabels = getLabelPaths(valid)
    testLabels = getLabelPaths(test)

    trainDir = Path(f"datasets/processed/{dirName}/images/train")
    validDir = Path(f"datasets/processed/{dirName}/images/valid")
    testDir = Path(f"datasets/processed/{dirName}/images/test")
    shutil.rmtree(trainDir, ignore_errors=True)
    shutil.rmtree(validDir, ignore_errors=True)
    shutil.rmtree(testDir, ignore_errors=True)

    copyFiles(trainLabels, "train")
    copyFiles(validLabels, "valid")
    copyFiles(testLabels, "test")
    copyFiles(train.astype(str), "train")
    copyFiles(valid.astype(str), "valid")
    copyFiles(test.astype(str), "test")

    trainSizes.append(len(train))
    validSizes.append(len(valid))
    testSizes.append(len(test))

for i in range(len(dirNames)):
    print(f"Directory Name:", dirNames[i])
    print(
        f"""\
    Test Size: {testSizes[i]} ({round(testSizes[i]/sum(testSizes)*100)}%)
    Validation Size: {validSizes[i]} ({round(validSizes[i]/sum(validSizes)*100)}%)
    Training Size: {trainSizes[i]} ({round(trainSizes[i]/sum(trainSizes)*100)}%)
    """
    )

print("Total:")
print(
    f"""\
    Test Size: {sum(testSizes)}
    Validation Size: {sum(validSizes)}
    Training Size: {sum(trainSizes)}
    """
)

Directory Name: old-bags
    Test Size: 65 (100%)
    Validation Size: 130 (4%)
    Training Size: 458 (3%)
    
Directory Name: icbj-v2i-yolov8-cvat
    Test Size: 0 (0%)
    Validation Size: 16 (0%)
    Training Size: 64 (0%)
    
Directory Name: gate-jedy8
    Test Size: 0 (0%)
    Validation Size: 243 (7%)
    Training Size: 974 (7%)
    
Directory Name: auv-project-gate-sample
    Test Size: 0 (0%)
    Validation Size: 87 (2%)
    Training Size: 348 (2%)
    
Directory Name: auv-project-quali-gate
    Test Size: 0 (0%)
    Validation Size: 235 (6%)
    Training Size: 943 (6%)
    
Directory Name: input
    Test Size: 0 (0%)
    Validation Size: 70 (2%)
    Training Size: 283 (2%)
    
Directory Name: sauvc_v2_bboxes_yolo_front_cvat
    Test Size: 0 (0%)
    Validation Size: 152 (4%)
    Training Size: 608 (4%)
    
Directory Name: pooltests/271223/main_gate_detection
    Test Size: 0 (0%)
    Validation Size: 178 (5%)
    Training Size: 716 (5%)
    
Directory Name: pooltests/0401

### Print Paths for YAML File

In [ ]:
misdetection_train_paths = [
    str(Path(*path.parts[2:]) / "images/train") for path in misdetection_dir_paths
]
for path in natsorted(misdetection_train_paths):
    print(path, end=",\n")

In [ ]:
misdetection_valid_paths = [
    str(Path(*path.parts[2:]) / "images/valid") for path in misdetection_dir_paths
]
for path in natsorted(misdetection_valid_paths):
    print(path, end=",\n")